# Case Study 1 : Collecting Data from Twitter

Due Date: 1/27/2020, **BEFORE the beginning of class at 11:00am**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    Irean Ali
    
    Achu Balasubramanian
    
    Josh Lovering
    
    ...

**Suggested Readings:** 
* Chapter 1 and Chapter 9 of the book "Mining the Social Web" can help a lot if you get stuck. 
* In fact, it is intentional that many of these questions can be answered directly from there (except for question 4)!
* The idea is to ease you into the case studies :-)

**Don't forget!**
* You will need to install the python-twitter library to access the Twitter API
 * pip install twitter


** NOTE **
* **Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost**.

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [69]:
import twitter
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = "QoTiAk8g8Hm9Zhejyt1GxF9Bp"
    CONSUMER_SECRET = "Lj6Hg1XlDcKeT2pFNS330vN7RqgRzxHpsUovESMd9braCZ4WGO"
    # Open Authorizan
    OAUTH_TOKEN = "571213367-dPdnChuVRjwRHoqfH0pi15IbgGhJmRoKxJjrP9wD"
    OAUTH_TOKEN_SECRET = "3cp8koRc8EKcVn3DkI0HoGquXJWMnAYgU7pzMiMnNq3jq"
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api
print(twitter_api)
twitter_api = oauth_login()
print(twitter_api)
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


In [70]:
WORLD_WOE_ID = 1
#Where on earth - entire world
US_WOE_ID = 23424977
#Where on earth - United States
world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
#creates listt of Python dictionaries from API
us_trends = twitter_api.trends.place(_id=US_WOE_ID)
print(world_trends)
print()
print(us_trends)


[{'trends': [{'name': '#MLKDay', 'url': 'http://twitter.com/search?q=%23MLKDay', 'promoted_content': None, 'query': '%23MLKDay', 'tweet_volume': 125842}, {'name': '#OTDirecto20E', 'url': 'http://twitter.com/search?q=%23OTDirecto20E', 'promoted_content': None, 'query': '%23OTDirecto20E', 'tweet_volume': 113362}, {'name': '#BlueMonday', 'url': 'http://twitter.com/search?q=%23BlueMonday', 'promoted_content': None, 'query': '%23BlueMonday', 'tweet_volume': 110684}, {'name': 'Jadine', 'url': 'http://twitter.com/search?q=Jadine', 'promoted_content': None, 'query': 'Jadine', 'tweet_volume': 29761}, {'name': '#Missguided', 'url': 'http://twitter.com/search?q=%23Missguided', 'promoted_content': None, 'query': '%23Missguided', 'tweet_volume': 10538}, {'name': '#AlwaysWithYoon', 'url': 'http://twitter.com/search?q=%23AlwaysWithYoon', 'promoted_content': None, 'query': '%23AlwaysWithYoon', 'tweet_volume': 31383}, {'name': 'SideM', 'url': 'http://twitter.com/search?q=SideM', 'promoted_content': Non

In [71]:
import json

print(json.dumps(world_trends, indent=1))
#dumps dictionarys into json format
print()
print (json.dumps(us_trends, indent=1))

[
 {
  "trends": [
   {
    "name": "#MLKDay",
    "url": "http://twitter.com/search?q=%23MLKDay",
    "promoted_content": null,
    "query": "%23MLKDay",
    "tweet_volume": 125842
   },
   {
    "name": "#OTDirecto20E",
    "url": "http://twitter.com/search?q=%23OTDirecto20E",
    "promoted_content": null,
    "query": "%23OTDirecto20E",
    "tweet_volume": 113362
   },
   {
    "name": "#BlueMonday",
    "url": "http://twitter.com/search?q=%23BlueMonday",
    "promoted_content": null,
    "query": "%23BlueMonday",
    "tweet_volume": 110684
   },
   {
    "name": "Jadine",
    "url": "http://twitter.com/search?q=Jadine",
    "promoted_content": null,
    "query": "Jadine",
    "tweet_volume": 29761
   },
   {
    "name": "#Missguided",
    "url": "http://twitter.com/search?q=%23Missguided",
    "promoted_content": null,
    "query": "%23Missguided",
    "tweet_volume": 10538
   },
   {
    "name": "#AlwaysWithYoon",
    "url": "http://twitter.com/search?q=%23AlwaysWithYoon",
    "pr

In [117]:
query = 'Liverpool' #chosen trend to search live tweets

count = 100 #number of recent tweets per page

search_results = twitter_api.search.tweets(q=query, count=count, result_types='recent')

statuses = search_results['statuses']
#iterates through batch of 300 tweets

#iterates through 5 more batches
for _ in range(5):
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except(KeyError): # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Show one sample search result by slicing the list...
print(json.dumps(statuses[0], indent=1))



Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500
{
 "created_at": "Mon Jan 20 16:09:21 +0000 2020",
 "id": 1219290828264366080,
 "id_str": "1219290828264366080",
 "text": "RT @footballdaily: Gary Neville hating his life when Salah makes it 2-0 will make a Liverpool fans day https://t.co/bmFJgPd5Ib",
 "truncated": false,
 "entities": {
  "hashtags": [],
  "symbols": [],
  "user_mentions": [
   {
    "screen_name": "footballdaily",
    "name": "Football Daily",
    "id": 278038673,
    "id_str": "278038673",
    "indices": [
     3,
     17
    ]
   }
  ],
  "urls": [],
  "media": [
   {
    "id": 1219227761014968326,
    "id_str": "1219227761014968326",
    "indices": [
     103,
     126
    ],
    "media_url": "http://pbs.twimg.com/media/EOuRXs_XUAEMD2l.jpg",
    "media_url_https": "https://pbs.twimg.com/media/EOuRXs_XUAEMD2l.jpg",
    "url": "https://t.co/bmFJgPd5Ib",
    "display_url": "pic.twitter.com/bmFJgPd5Ib",
  

In [84]:
len(search_results['statuses'])

100

In [93]:
len(search_results)

2

In [99]:
#print(search_results)

In [95]:
print(list(search_results.keys()))

['statuses', 'search_metadata']


In [90]:
len(next_results)

68

In [97]:
print(next_results)

?max_id=1219283836435083264&q=Liverpool&count=100&include_entities=1


In [119]:
len(statuses)

600

In [120]:
#store the tweets
with open('LiverpoolTweetsRecent.json', 'w') as jsonfile:
    json.dump(statuses, jsonfile, indent=1)

### Report some statistics about the tweets you collected 

* The topic of interest: Liverpool


* The total number of tweets collected: 600

*-----------------------

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [121]:
#----------------------------------------------
with open('LiverpoolTweetsRecent.json', 'r') as jsonfile:
    statuses_loaded = json.load(jsonfile)

#extracting text from the tweets
status_texts = [ status['text'] 
                 for status in statuses_loaded ]
#extracting screen names from the tweets
screen_names = [ user_mention['screen_name'] 
                 for status in statuses_loaded
                     for user_mention in status['entities']['user_mentions'] ]
#extracting hashtags from the tweets
hashtags = [ hashtag['text'] 
             for status in statuses_loaded
                 for hashtag in status['entities']['hashtags'] ]
#collect all words from all of the tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]
#print first 5 items for text and words
print(json.dumps(status_texts[0:5], indent=1))
print(json.dumps(words[0:5], indent=1))


[
 "RT @footballdaily: Gary Neville hating his life when Salah makes it 2-0 will make a Liverpool fans day https://t.co/bmFJgPd5Ib",
 "@UTDSean27 @PassionOx @nikhillmishra @A_Martial_9_Utd @AdilAkthar1 @ManLikeAWB @Lecantona7 Jus cus you got a light\u2026 https://t.co/TBQjRIoQ1V",
 "@Ay_Miles @holuwafemmie The club is ruined already. Liverpool and Man City are coming to take the revenue record from them.",
 "RT @JanAageFjortoft: Timo Werner to Liverpool!!!!! \n\nI can see it happens. Reports in Germany today also support that.\n\nWerner will love to\u2026",
 "RT @VarskySports: Liverpool no para: tach\u00f3 2-0 a Manchester United con goles de Van Dijk-Salah\n22 PJ\n21 victorias\n1 empate\n64/66 pts en dis\u2026"
]
[
 "RT",
 "@footballdaily:",
 "Gary",
 "Neville",
 "hating"
]


In [136]:
from collections import Counter
#creates frequency distribution from all of the words
# for item in [words]:
#     c = Counter(item)
#     print(c.most_common()[:30]) # top 10
#     print()

from prettytable import PrettyTable
# display tuples of words and count of words in a table using PrettyTable
for label, data in (('Word', words),):
    #for each word in words
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common(30) ]
    #add row with word and count
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print (pt)

+-----------+-------+
| Word      | Count |
+-----------+-------+
| RT        |   393 |
| Liverpool |   392 |
| the       |   230 |
| to        |   210 |
| a         |   191 |
| in        |   150 |
| this      |    97 |
| is        |    92 |
| of        |    78 |
| and       |    72 |
| have      |    69 |
| -         |    66 |
| are       |    64 |
| Werner    |    61 |
| that      |    55 |
| .         |    55 |
| Timo      |    54 |
| for       |    50 |
| team      |    46 |
| I         |    43 |
| at        |    43 |
| de        |    42 |
| just      |    42 |
| The       |    41 |
| United    |    38 |
| it        |    37 |
| with      |    37 |
| Premier   |    37 |
| be        |    36 |
| on        |    35 |
+-----------+-------+


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [123]:
#----------------------------------------------
retweets = [
            # Store out a tuple of these three values ...
            (status['retweet_count'], 
             status['retweeted_status']['user']['screen_name'],
             status['text']) 
            
            # ... for each status ...
            for status in statuses_loaded 
            
            # ... so long as the status meets this condition.
                if 'retweeted_status' in status
           ]

# Slice off the first 10 from the sorted results and display 
# each item in the tuple

pt = PrettyTable(field_names=['Count', 'Screen Name', 'Text'])
[ pt.add_row(row) for row in sorted(retweets, reverse=True)[:10] ]
# add a row containing the next tuple
pt.max_width['Text'] = 50
pt.align= 'l'
print(pt)

+-------+---------------+----------------------------------------------------+
| Count | Screen Name   | Text                                               |
+-------+---------------+----------------------------------------------------+
| 6515  | goal          | RT @goal: VAR: Rules out Liverpool goal            |
|       |               |                                                    |
|       |               | Linesman: Rules out Liverpool goal                 |
|       |               |                                                    |
|       |               | United fans:                                       |
| 6305  | brfootball    | RT @brfootball: 1) Liverpool—64                    |
|       |               | .                                                  |
|       |               | .                                                  |
|       |               | .                                                  |
|       |               | .                         

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [132]:
x = ["red", "blue", "red", "pink"]
c = Counter(x)
print(c)
print(c.most_common(1))
for k in y.keys():
    print(k)

Counter({'red': 2, 'blue': 1, 'pink': 1})
[('red', 2)]
charlu
fdsf


In [134]:
#----------------------------------------------
from prettytable import PrettyTable

#Display tuples of (screen names, counts) and (hashtags, counts)
for label, data in ( 
                    ('Screen Name', screen_names), 
                    ('Hashtag', hashtags)):
    #for each name in screen_names and hashtag in hashtags
    #add a new row with the tuple displayed in the table
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [pt.add_row(kv) for kv in c.most_common(10)]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print(pt)

+-----------------+-------+
| Screen Name     | Count |
+-----------------+-------+
| DeadlineDayLive |    24 |
| goal            |    18 |
| premierleague   |    12 |
| JanAageFjortoft |    11 |
| AnfieldWatch    |    11 |
| footballdaily   |     9 |
| OfficialFPL     |     9 |
| JayLleweIIyn    |     8 |
| Linda_Pizzuti   |     8 |
| AnfieldEdition  |     8 |
+-----------------+-------+
+----------------+-------+
| Hashtag        | Count |
+----------------+-------+
| Liverpool      |     5 |
| LFC            |     5 |
| FPL            |     5 |
| LIVMUN         |     5 |
| MUFC           |     3 |
| PL             |     3 |
| YNWA           |     3 |
| Video          |     2 |
| lfc            |     2 |
| theunbearables |     2 |
+----------------+-------+


* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [21]:
import sys
import time
from urllib.request import URLError
from http.client import BadStatusLine
import json
import twitter

def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw): 
    
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
    
        if wait_period > 3600: # Seconds
            print >> sys.stderr, 'Too many retries. Quitting.'
            raise e
    
        # See https://dev.twitter.com/docs/error-codes-responses for common codes
    
        if e.e.code == 401:
            print >> sys.stderr, 'Encountered 401 Error (Not Authorized)'
            return None
        elif e.e.code == 404:
            print >> sys.stderr, 'Encountered 404 Error (Not Found)'
            return None
        elif e.e.code == 429: 
            print >> sys.stderr, 'Encountered 429 Error (Rate Limit Exceeded)'
            if sleep_when_rate_limited:
                print >> sys.stderr, "Retrying in 15 minutes...ZzZ..."
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print >> sys.stderr, '...ZzZ...Awake now and trying again.'
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print >> sys.stderr, 'Encountered %i Error. Retrying in %i seconds' % \
                (e.e.code, wait_period)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function
    
    wait_period = 2 
    error_count = 0 

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except(twitter.api.TwitterHTTPError, e):
            error_count = 0 
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except(URLError, e):
            error_count += 1
            print >> sys.stderr, "URLError encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise
        except(BadStatusLine, e):
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise

# Sample usage

twitter_api = oauth_login()

# See https://dev.twitter.com/docs/api/1.1/get/users/lookup for 
# twitter_api.users.lookup

response = make_twitter_request(twitter_api.users.lookup, 
                                screen_name="ladygaga")

print(json.dumps(response, indent=1))

[
 {
  "id": 14230524,
  "id_str": "14230524",
  "name": "Lady Gaga",
  "screen_name": "ladygaga",
  "location": "",
  "description": "",
  "url": "https://t.co/6CDfvJPhW0",
  "entities": {
   "url": {
    "urls": [
     {
      "url": "https://t.co/6CDfvJPhW0",
      "expanded_url": "http://ladygaga.com",
      "display_url": "ladygaga.com",
      "indices": [
       0,
       23
      ]
     }
    ]
   },
   "description": {
    "urls": []
   }
  },
  "protected": false,
  "followers_count": 80688439,
  "friends_count": 123749,
  "listed_count": 217738,
  "created_at": "Wed Mar 26 22:37:48 +0000 2008",
  "favourites_count": 2071,
  "utc_offset": null,
  "time_zone": null,
  "geo_enabled": false,
  "verified": true,
  "statuses_count": 9051,
  "lang": null,
  "status": {
   "created_at": "Fri Jan 17 18:32:13 +0000 2020",
   "id": 1218239620330086405,
   "id_str": "1218239620330086405",
   "text": "RT @hauslabs: ENTER NOW via our latest post on Instagram #HAUSCALL &gt; https://t.co/b0K

In [142]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

from functools import partial
from sys import maxsize

def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                              friends_limit=maxsize, followers_limit=maxsize):
    
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids, 
                              count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids, 
                                count=5000)

    friends_ids, followers_ids = [], []
    
    for twitter_api_func, limit, ids, label in [
                    [get_friends_ids, friends_limit, friends_ids, "friends"], 
                    [get_followers_ids, followers_limit, followers_ids, "followers"]
                ]:
        
        if limit == 0: continue
        
        cursor = -1
        while cursor != 0:
        
            # Use make_twitter_request via the partially bound callable...
            if screen_name: 
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
        
#             print >> sys.stderr, 'Fetched {0} total {1} ids for {2}'.format(len(ids), 
#                                                     label, (user_id or screen_name))
            print(f"Fetched {len(ids)} total {label} ids for {user_id or screen_name}")
        
            # XXX: You may want to store data during each iteration to provide an 
            # an additional layer of protection from exceptional circumstances
        
            if len(ids) >= limit or response is None:
                break

    # Do something useful with the IDs, like store them to disk...
    return friends_ids[:friends_limit], followers_ids[:followers_limit]
    # Sample usage

twitter_api = oauth_login()

friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       screen_name="BrandSanderson", 
                                                       friends_limit=10000, 
                                                       followers_limit=10000)

print(friends_ids)
print(followers_ids)


Fetched 240 total friends ids for BrandSanderson
Fetched 5000 total followers ids for BrandSanderson
Fetched 10000 total followers ids for BrandSanderson
[348278208, 819907370, 1745818897, 131631056, 167384955, 741103986, 31326445, 61879392, 15697107, 63242060, 126803141, 284175753, 11348282, 45129669, 108740062, 26029878, 9507172, 161956320, 117521574, 1118034042, 28620202, 2511947989, 302818148, 130241174, 512434318, 14104488, 382731339, 19783965, 61715911, 12080652, 3114744590, 1245115550, 14446025, 18093012, 107395417, 38036633, 356180427, 14924430, 1733293656, 17790274, 20251800, 206611680, 14368074, 22520465, 26560483, 13598222, 17425830, 46868950, 18959480, 15447034, 1275701432, 228047564, 960860563, 5848412, 64496398, 2820491389, 204870137, 210684204, 2557037323, 2718430589, 2176097712, 8830022, 320698518, 1243725331, 52731757, 33439720, 108508251, 2421065262, 475645133, 473604055, 8952022, 19041142, 37498067, 221088211, 1912392325, 14416749, 127476066, 14426048, 17219386, 5195

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [6]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary


*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [9]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 15 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu).

#### We auto-process the submissions so make sure your subject line is *exactly*:

### DS3010 Case Study 1 Team ??

#### where ?? is your team number.
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 100 **


---------------------------------------------------------------------------
** Notebook results:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.





---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

